In [3]:
open Core

Ocaml最棒的特性之一是，它提供了一个简洁而且表达能力很强的系统来声明新的数据类型，记录则是这个系统中一个关键的要素

需要说明，记录字段名必须以一个小写的子母开头

In [8]:
type host_info = 
{hostname: string;
os_name:string;
cpu_arch:string;
timestamp:Time.t;
}

type host_info = {
  hostname : string;
  os_name : string;
  cpu_arch : string;
  timestamp : Core.Time.t;
}


In [9]:
# require "core_extended" 

No such package: core_extended


In [7]:
open Core_extended


error: compile_error

In [10]:
let my_host = 
{
hostname = "hostname";
os_name = "uname -s";
cpu_arch = "uname -p";
timestamp = Time.now();
}

val my_host : host_info =
  {hostname = "hostname"; os_name = "uname -s"; cpu_arch = "uname -p";
   timestamp = 2018-01-27 09:09:12.376816-08:00}


你可能想知道编译器如何推断出`my_host`类型是`host_info`。在这里，编译器确定类型的关键是记录字段名。在这一张后面，我们还会讨论多个记录类型有相同的字段名时会发生什么

可以使用点击法来抽取元素

In [12]:
my_host.cpu_arch;;

- : string = "uname -p"
